![](https://i.imgur.com/ftKr4IX.png)

# BTH004 Assignment1 Experiment Report
* Name: Zhu Yun
* Student ID: 201619630425
* Teacher: Chen Zhenbo
* Problem: multiple knapsack problem
* Date: 2018/10/23
---

## Problem description:The multiple knapsack problem
![](https://i.imgur.com/7NiGHbw.png)

## Part 1 - Greedy and neighborhood search algorithms
### A (contructive) greedy algorithm.
#### 1. Algorithm description
![](https://i.imgur.com/pepCZNi.png)

#### 2. The rough flowchart about how to calculate the result(The details I will explain in the next section):
![](https://i.imgur.com/1tdJNVl.png)

#### 4. The details about implementing this algorithm
* About how to choose the order of knapsacks
    * I use the biggest knapsack at first, when the knapsack can not hold any stuff. I will use the second biggest...
* About how to choose stuff
    * The biggest value(v/w) of stuff will be put into knapsack at first, if the weight of the stuff is larger than the capacity of knapsack, I ignore this stuff and select next one(still jugde the weight).
---

#### 5.The source codes are presented below.

In [3]:
# This line magic function makes it possible to see all results after running this algorithm.
%config ZMQInteractiveShell.ast_node_interactivity='all'

In [39]:
def greedy_solution(values: "The values of stuff(a list of integers)", 
                    weights: "The weights of stuff(a list of integers)", 
                    capacities: "The capacities of knapsack(a list of integers)")->"function return should be a integer":
    
    assert len(weights) == len(values), "One weight corresopnds to one value"
    
    # record the v/w
    gravities = list(map(lambda x, y: x/y, values, weights))
    
    # record the stuff if is used
    excludedStuff = []
    for i in range(len(weights)):
        excludedStuff.append((gravities[i], weights[i], values[i]))

    # sort the list
    excludedStuff = sorted(excludedStuff)

    result = 0
    # record which of the index should be used.
    priority = 0
    # sort the capacities, in order to let the biggest bag be used firstly.
    capacities = sorted(capacities)
    while excludedStuff and capacities:
        capacity = capacities.pop()
        priority = len(excludedStuff) - 1
        while capacity > 0 and priority >= 0:
            stuff = excludedStuff[priority]
            if(capacity < stuff[1]):
                priority -= 1
            else:
                capacity -= stuff[1]
                result += stuff[2]
                # when the stuff is used, we remove it from excludedStuff.
                re = excludedStuff.pop(priority)
                # reset the priority(minus one)
                priority -= 1
    return result

#### 6. Test cases：
<table border="1">
  <tr>
    <th>Values</th>
    <th>Weights</th>
    <th>Capacity</th>
    <th>Result</th>
  </tr>
  <tr>
    <td>[4, 10, 5, 8]</td>
    <td>[3, 9, 4, 6]</td>
      <td>[12]</td>
      <td>12</td>
  </tr>
    <tr>
    <td>[4, 10, 5, 8]</td>
    <td>[3, 9, 4, 6]</td>
      <td>[12, 5]</td>
      <td>17</td>
  </tr>
    <tr>
    <td>[8, 3, 2]</td>
    <td>[8, 5, 3]</td>
      <td>[13, 3]</td>
      <td>10</td>
  </tr>
    <tr>
    <td>[10, 8, 1, 5]</td>
    <td>[10, 7, 4, 8]</td>
      <td>[14, 8]</td>
      <td>14</td>
  </tr>
</table>

In [40]:
greedy_solution([4, 10, 5, 8], [3, 9, 4, 6], [12])
greedy_solution([4, 10, 5, 8], [3, 9, 4, 6], [12, 5])
greedy_solution([8, 3, 2], [8, 5, 3], [13, 3])
greedy_solution([10, 8, 1, 5], [10, 7, 4, 8], [14, 8])

12

17

10

14

* As we can see, the answers of these quesions are all correct.

---

### An improving search (neighborhood search) algorithm.
#### 1. Algorithm description
![](https://i.imgur.com/LrYeMpG.png)

---

##### 1.1 The graphic description of this algorithm
* In order to understand it easily, we use very simple sample.(Maybe it can not describe the algorithm very clearly)
![](https://i.imgur.com/2pTglXY.png)

#### 2. The details
* How to choose the start position
    * I use the result calculated by greedy algorithm.
* **How to choose the neighbours**
    1. Select one stuff from outside.  
    2. use nested for-loop  
    3. for center_bag in allBags:  
        * for other_bag in allBags:
          * remove one stuff from center_bag in order to hold selected stuff which is from outside
          * remove one stuff from other_bag in order to hold removed stuff from center_bag
    * we get a result, if it is bigger than before we set it as a new starter and restart this nested-loop. else we end it, return the previousresult`

##### 2.1 The more details of how to choose neighbours is pictured below.
![](https://i.imgur.com/bU7Qo0y.png)

#### 3. The source codes:

In [19]:
import copy


# calculate the amount of values
def get_total_val(allBags):
    res = 0
    for bag in allBags:
        if bag:
            for item in bag:
                res += item[2]
    return res


def ns_solution(values: "The values of stuff(a list of integers)",
                  weights: "The weights of stuff(a list of integers)",
                  capacitiesInvar: "The capacities of knapsack(a list of integers)") -> "function return should be a integer":
    # ensure the length of weights equals to the length of values
    assert len(weights) == len(values), "One weight corresopnds to one value"

    # the original capacity of each bag
    # I decide to use the biggest bag firstly.
    capacitiesInvar = sorted(capacitiesInvar)
    capacities = copy.deepcopy(capacitiesInvar)

    # record the v/w
    gravities = list(map(lambda x, y: x / y, values, weights))

    # record the stuff that is not used
    excludedStuff = []
    # initialize the excludedStuff(add all at first)
    for i in range(len(weights)):
        excludedStuff.append((gravities[i], weights[i], values[i]))

    # sort the list(according to the v/w, the bigger value(v/w) is in the tail) in order to make work easily
    excludedStuff = sorted(excludedStuff)
    # record the answer
    result = 0
    # record which of the index should be used.
    priority = 0

    # record the information of all stuff in bags
    allBags = [[] for i in range(len(capacities))]
    # record the index of which bag is used
    # the biggest bag in first, the second biggest bag in second...
    # the format like [[bagA], [(1.3, 4, 3), (1.2, 3, 3), ...], [bagC]...]
    idx = -1
    storedCapacities = []
    iteraNum = 0
    while excludedStuff and capacities:
        tmpOtherStuff = None
        tmpCenterStuff = None
        # use the biggest bag at first.
        capacity = capacities.pop()
        idx += 1
        # when restart a loop, we set priority as len(excludedStuff) - 1, in oreder to iterate all stuff in excludedStuff
        priority = len(excludedStuff) - 1
        while capacity > 0 and priority >= 0:
            stuff = excludedStuff[priority]
            # if the stuff is heavier than the capacity of bag, we skip this stuff.
            if (capacity < stuff[1]):
                priority -= 1

            else:
                capacity -= stuff[1]
                result += stuff[2]

                allBags[idx].append(stuff)
                # when the stuff is used, we remove it from excludedStuff.
                excludedStuff.pop(priority)
                # reset the priority(minus one)
                priority -= 1
        storedCapacities.append(capacity)

    # if there is no stuff left, we return the result.
    if not excludedStuff:
        return result

    # ==============Let's start neighbour search========================
    print("x(0):", result)
    # iterate select stuff from one bag into another
    currentRes = result
    maxRes = result
    # store better strategy
    betterBags = copy.deepcopy(allBags)

    betterCapacities = copy.deepcopy(storedCapacities)
    betterExcludedStuff = copy.deepcopy(excludedStuff)
    nsExcludedStuff = copy.deepcopy(excludedStuff)
    nsAllBags = copy.deepcopy(allBags)
    nsCapacities = copy.deepcopy(storedCapacities)
    # if there exists at least, we can use this way. When previous value is smaller than all his neighbors, we stop it.
    while True and nsExcludedStuff:

        eStuff = nsExcludedStuff.pop()
        # iterate all the bags
        for centerBag in nsAllBags:
            if not centerBag:
                # The state is end state. If it is empty, no stuff can be stored in this bag. So we just skip this bag.
                continue
            else:
                # record the index of centerBag in the allBags, it will help us to calculate its room
                cenIdx = nsAllBags.index(centerBag)
                # record the num of iteration
                cenIterNum = 0
                m = len(centerBag)

                

                # Take a suitable item from the centerBag
                while cenIterNum < m:
                    cenStuff = centerBag.pop()
                    if (nsCapacities[cenIdx] + cenStuff[1]) >= eStuff[1]:
                        nsCapacities[cenIdx] += cenStuff[1] - eStuff[1]
                        centerBag.append(eStuff)

                        break
                    else:
                        centerBag.insert(0, cenStuff)
                        cenIterNum += 1
                # If there is no suitable item，we choose the next centerBag, skip this loop
                if cenIterNum == m:
                    
                    break
            for otherBag in nsAllBags:
                otherIdx = nsAllBags.index(otherBag)
                # if the centerBag or otherBag is empty, we skip it.
                if centerBag == otherBag:
                    continue
                if not otherBag:
                    if nsCapacities[otherIdx] >= cenStuff[1]:
                        otherBag.append(cenStuff)
                        nsCapacities[otherIdx] -= cenStuff[1]
                        currentRes = get_total_val(nsAllBags)
                        if currentRes > maxRes:
                            maxRes = currentRes
                            betterBags[otherIdx] = otherBag
                            betterBags[cenIdx] = centerBag
                            betterExcludedStuff = nsExcludedStuff
                            betterCapacities = nsCapacities
                            break
                    else:
                        tmpCenterStuff = cenStuff

                else:

                    otherIterNum = 0
                    n = len(otherBag)
                    
                    # we remove one stuff from otherBag(if it is not ok, we put it back) until it can hold the stuff from centerBag
                    while otherIterNum < n:
                        otherStuff = otherBag.pop()
                        if nsCapacities[otherIdx] + otherStuff[1] >= cenStuff[1]:
                            otherBag.append(cenStuff)
                            nsCapacities[otherIdx] += otherStuff[1] - cenStuff[1]
                            #                             currentRes = maxRes - otherStuff[1] + eStuff[1]
                            currentRes = get_total_val(nsAllBags)
                            if currentRes > maxRes:
                                tmpOtherStuff = otherStuff
                                maxRes = currentRes
                                betterBags[otherIdx] = otherBag
                                betterBags[cenIdx] = centerBag
                                betterExcludedStuff = nsExcludedStuff
                                betterCapacities = nsCapacities
                            break
                        else:
                            # otherBag.append(otherStuff)
                            otherBag.insert(0, otherStuff)
                            otherIterNum += 1
                            if otherIterNum == n:
                                tmpCenterStuff = cenStuff
                    currentRes = get_total_val(nsAllBags)
                    if currentRes > maxRes:
                        maxRes = currentRes
                        betterBags[otherIdx] = otherBag
                        betterBags[cenIdx] = centerBag
                        betterExcludedStuff = nsExcludedStuff
                        betterCapacities = nsCapacities
        print("The max value of Negihbors(x(%d)) is %d"%(iteraNum, maxRes))
        if maxRes > result:
            if tmpOtherStuff:
                betterExcludedStuff.insert(0, tmpOtherStuff)
                
            if tmpCenterStuff:
                betterExcludedStuff.insert(0, tmpCenterStuff)
            result = maxRes
            nsAllBags = copy.deepcopy(betterBags)
            nsCapacities = copy.deepcopy(betterCapacities)
            nsExcludedStuff = copy.deepcopy(betterExcludedStuff)
            iteraNum += 1
            print("x(%d): %d"%(iteraNum, result))
        else:
            print("Because it is smaller than or equals to x(%d): %d, so we end it."%(iteraNum, result))
            break

    return maxRes

#### 4. Test cases：
<table border="1">
  <tr>
    <th>Values</th>
    <th>Weights</th>
    <th>Capacity</th>
    <th>Result</th>
  </tr>
  <tr>
    <td>[4, 10, 5, 8]</td>
    <td>[3, 9, 4, 6]</td>
      <td>[12, 5]</td>
      <td>19</td>
  </tr>
    <tr>
    <td>[8, 3, 2]</td>
    <td>[8, 5, 3]</td>
      <td>[13, 3]</td>
      <td>13</td>
  </tr>
    <tr>
    <td>[10, 8, 1, 5]</td>
    <td>[10, 7, 4, 8]</td>
      <td>[14, 8]</td>
      <td>19</td>
  </tr>
    <tr>
    <td>[11, 8, 1, 5, 10]</td>
    <td>[10, 7, 4, 8, 9]</td>
      <td>[14, 8]</td>
      <td>20</td>
  </tr>
    
</table>

In [20]:
ns_solution([4, 10, 5, 8], [3, 9, 4, 6], [12, 5])
print('----------------------------------')
ns_solution([8, 3, 2], [8, 5, 3], [13, 3])
print('----------------------------------')
ns_solution([10, 8, 1, 5], [10, 7, 4, 8], [14, 8])
print('----------------------------------')
ns_solution([11, 8, 1, 5, 10], [10, 7, 4, 8, 9], [14, 8])
print('----------------------------------')

x(0): 17
The max value of Negihbors(x(0)) is 19
x(1): 19
The max value of Negihbors(x(1)) is 19
Because it is smaller than or equals to x(1): 19, so we end it.


19

----------------------------------
x(0): 10
The max value of Negihbors(x(0)) is 13
x(1): 13


13

----------------------------------
x(0): 14
The max value of Negihbors(x(0)) is 19
x(1): 19
The max value of Negihbors(x(1)) is 19
Because it is smaller than or equals to x(1): 19, so we end it.


19

----------------------------------
x(0): 14
The max value of Negihbors(x(0)) is 19
x(1): 19
The max value of Negihbors(x(1)) is 20
x(2): 20
The max value of Negihbors(x(2)) is 20
Because it is smaller than or equals to x(2): 20, so we end it.


20

----------------------------------


* As we can see, the answers are all right.

---

## Part 2 - Tabu-search

#### 1. Algorithm description
A problem with the neighborhood search heuristics is that they eventually will get stuck in a locally optimal solution. A locally optimal solution is a solution that cannot be improved by moving to any of the solutions that are sufficiently close in the search space (i.e., in the neighborhood), but there are solutions farther away that are better. The ideas of tabu-search methods, which are based on neighborhood search,
are:
* It is possible to move to a solution with worse objective if there is no
improving solution in the neighborhood (note that it is necessary to keep
track of the so far best found solution).
* The kmost recent solutions are tabu, i.e., they cannot be visited (otherwise
the algorithm might immediately go back to the locally optimal
solution).


#### 2. The flowchart
> I copy it from PPT
![](https://i.imgur.com/xL30MJU.png)

#### 3. The details
* It is similar with the neighbour algorithm. But it allows to move to worse case in order to find global optimal solution.
* The stuff removed from centerBag can not be stored in otherBag, I will put it into excludedBag.
* I will put the stuff removed from otherBag into excludedBag.
* Because some of bags maybe empty, so after putting stuff into these bags the excludedBag will be empty. So some examples I runned will interrupt before running all iterations.

#### 4. The source code.

In [192]:
import copy


# calculate the amount of values
def get_total_val(allBags):
    res = 0
    for bag in allBags:
        if bag:
            for item in bag:
                res += item[2]
    return res


def tabu_solution(values: "The values of stuff(a list of integers)",
                  weights: "The weights of stuff(a list of integers)",
                  capacitiesInvar: "The capacities of knapsack(a list of integers)",
                  numOfIter: "The value must be integer.") -> "function return should be a integer":
    # ensure the length of weights equals to the length of values
    assert len(weights) == len(values), "One weight corresopnds to one value"
    NUM_OF_ITER = numOfIter
    # the original capacity of each bag
    # I decide to use the biggest bag firstly.
    capacitiesInvar = sorted(capacitiesInvar)
    capacities = copy.deepcopy(capacitiesInvar)

    # record the v/w
    gravities = list(map(lambda x, y: x / y, values, weights))

    # record the stuff that is not used
    excludedStuff = []
    # initialize the excludedStuff(add all at first)
    for i in range(len(weights)):
        excludedStuff.append((gravities[i], weights[i], values[i]))

    # sort the list(according to the v/w, the bigger value(v/w) is in the tail) in order to make work easily
    excludedStuff = sorted(excludedStuff)
    # record the answer
    result = 0
    # record which of the index should be used.
    priority = 0

    # record the information of all stuff in bags
    allBags = [[] for i in range(len(capacities))]
    # record the index of which bag is used
    # the biggest bag in first, the second biggest bag in second...
    # the format like [[bagA], [(1.3, 4, 3), (1.2, 3, 3), ...], [bagC]...]
    idx = -1
    storedCapacities = []
    while excludedStuff and capacities:
        # use the biggest bag at first.
        capacity = capacities.pop()
        idx += 1
        # when restart a loop, we set priority as len(excludedStuff) - 1, in oreder to iterate all stuff in excludedStuff
        priority = len(excludedStuff) - 1
        while capacity > 0 and priority >= 0:
            stuff = excludedStuff[priority]
            # if the stuff is heavier than the capacity of bag, we skip this stuff.
            if (capacity < stuff[1]):
                priority -= 1

            else:
                capacity -= stuff[1]
                result += stuff[2]

                allBags[idx].append(stuff)
                # when the stuff is used, we remove it from excludedStuff.
                excludedStuff.pop(priority)
                # reset the priority(minus one)
                priority -= 1
        storedCapacities.append(capacity)

    # if there is no stuff left, we return the result.
    if not excludedStuff:
        return result

    # ==============Let's start tabu search========================

    # There I just store one element in tabu list.
    tabu = None
    tmpTabu = None
    # iterate select stuff from one bag into another
    currentRes = result
    maxRes = result
    maxValInIter = 0
    # store better strategy
    betterBags = copy.deepcopy(allBags)

    betterCapacities = copy.deepcopy(storedCapacities)
    betterExcludedStuff = copy.deepcopy(excludedStuff)
    nsExcludedStuff = copy.deepcopy(excludedStuff)
    nsAllBags = copy.deepcopy(allBags)
    nsCapacities = copy.deepcopy(storedCapacities)
    print("x(0) is %d"%(result))

    # if there exists at least, we can use this way. When previous value is smaller than all his neighbors, we stop it.
    while numOfIter > 0 and nsExcludedStuff:
        tmpOtherStuff = None
        tmpCenterStuff = None
        maxValInIter = 0
        eStuff = nsExcludedStuff.pop()
        # iterate all the bags
        for centerBag in nsAllBags:
            if not centerBag:
                # The state is end state. If it is empty, no stuff can be stored in this bag. So we just skip this bag.
                continue
            else:
                # record the index of centerBag in the allBags, it will help us to calculate its room
                cenIdx = nsAllBags.index(centerBag)
                # record the num of iteration
                cenIterNum = 0
                m = len(centerBag)
 
                while cenIterNum < m:
                    cenStuff = centerBag.pop()
                    if (nsCapacities[cenIdx] + cenStuff[1]) >= eStuff[1]:
                        nsCapacities[cenIdx] += cenStuff[1] - eStuff[1]
                        centerBag.append(eStuff)
                        break
                    else:
                        centerBag.insert(0, cenStuff)
                        cenIterNum += 1      
                if cenIterNum == m:
                    break
            for otherBag in nsAllBags:
                otherIdx = nsAllBags.index(otherBag)
                # if the centerBag or otherBag is empty, we skip it.
                if centerBag == otherBag:
                    continue
                if not otherBag:
                    if nsCapacities[otherIdx] >= cenStuff[1]:
                        otherBag.append(cenStuff)
                        nsCapacities[otherIdx] -= cenStuff[1]
                        
                        if tabu != nsAllBags:
                            currentRes = get_total_val(nsAllBags)
                            if currentRes > maxValInIter:
                                tmpTabu = nsAllBags
                                maxValInIter = currentRes
                                maxRes = maxRes if maxRes >= currentRes else currentRes 
                                betterBags[otherIdx] = otherBag
                                betterBags[cenIdx] = centerBag
                                betterExcludedStuff = nsExcludedStuff
                                betterCapacities = nsCapacities
                            # break
                    else:
                        tmpCenterStuff = cenStuff

                else:
                    otherIterNum = 0
                    n = len(otherBag)
                    
                    
                    # nsExcludedStuff.insert(0, otherStuff)
                    otherStuff = None
                    while otherIterNum < n:
                        otherStuff = otherBag.pop()
                        
                        if nsCapacities[otherIdx] + otherStuff[1] >= cenStuff[1]:
                            otherBag.append(cenStuff)
                            nsCapacities[otherIdx] += otherStuff[1] - cenStuff[1]
                            #                             currentRes = maxRes - otherStuff[1] + eStuff[1]
                            if tabu != nsAllBags:
#                                 nsExcludedStuff.insert(0, otherStuff)
                                currentRes = get_total_val(nsAllBags)
                                if currentRes > maxValInIter:
                                    
                                    tmpOtherStuff = otherStuff
                                    tmpTabu = nsAllBags
                                    maxValInIter = currentRes
                                    maxRes = maxRes if maxRes >= currentRes else currentRes 
                                    betterBags[otherIdx] = otherBag
                                    betterBags[cenIdx] = centerBag
                                    betterExcludedStuff = nsExcludedStuff
                                    betterCapacities = nsCapacities
                                break
                            else:
                                
                                # otherBag.append(otherStuff)
                                otherBag.insert(0, otherStuff)
                                otherIterNum += 1
                                if otherIterNum == n:
                                    tmpCenterStuff = cenStuff
                        else:
                            
                            otherBag.insert(0, otherStuff)
                            otherIterNum += 1
                            if otherIterNum == n:
                                tmpCenterStuff = cenStuff
                    currentRes = get_total_val(nsAllBags)
                    if currentRes > maxValInIter:
                        tmpTabu = nsAllBags
                        maxValInIter = currentRes
                        maxRes = maxRes if maxRes >= currentRes else currentRes 
                        betterBags[otherIdx] = otherBag
                        betterBags[cenIdx] = centerBag
                        betterExcludedStuff = nsExcludedStuff
                        betterCapacities = nsCapacities

        tabu = tmpTabu
        #result = maxRes
        if tmpOtherStuff:
            betterExcludedStuff.insert(0, tmpOtherStuff)
        if tmpCenterStuff:
            betterExcludedStuff.insert(0, tmpCenterStuff)
        nsAllBags = copy.deepcopy(betterBags)
        nsCapacities = copy.deepcopy(betterCapacities)
        nsExcludedStuff = copy.deepcopy(betterExcludedStuff)
        numOfIter -= 1
#         print(nsAllBags)
        print("After %d loop, the x(%d) is %d and the best result is %d" % (NUM_OF_ITER-numOfIter, NUM_OF_ITER-numOfIter, maxValInIter, maxRes))

    return maxRes

#### 4. Test cases：
<table border="1">
  <tr>
    <th>Values</th>
    <th>Weights</th>
    <th>Capacity</th>
    <th>Result</th>
  </tr>
  <tr>
    <td>[4, 10, 5, 8]</td>
    <td>[3, 9, 4, 6]</td>
      <td>[12, 5]</td>
      <td>19</td>
  </tr>
    <tr>
    <td>[8, 3, 2]</td>
    <td>[8, 5, 3]</td>
      <td>[13, 3]</td>
      <td>13</td>
  </tr>
    <tr>
    <td>[10, 8, 1, 5]</td>
    <td>[10, 7, 4, 8]</td>
      <td>[14, 8]</td>
      <td>19</td>
  </tr>
    <tr>
    <td>[11, 8, 1, 5, 10]</td>
    <td>[10, 7, 4, 8, 9]</td>
      <td>[14, 8]</td>
      <td>20</td>
  </tr>
    
</table>

In [197]:
tabu_solution([4, 10, 5, 8], [3, 9, 4, 6], [12, 5], 2)
tabu_solution([8, 3, 2], [8, 5, 3], [13, 3],4)
tabu_solution([10, 8, 1, 5], [10, 7, 4, 8], [14, 8],4)
tabu_solution([11, 8, 1, 5, 10], [10, 7, 4, 8, 9], [14, 8],4)

x(0) is 17
After 1 loop, the x(1) is 19 and the best result is 19
After 2 loop, the x(2) is 17 and the best result is 19


19

x(0) is 10
After 1 loop, the x(1) is 13 and the best result is 13


13

x(0) is 14
After 1 loop, the x(1) is 19 and the best result is 19
After 2 loop, the x(2) is 14 and the best result is 19
After 3 loop, the x(3) is 19 and the best result is 19
After 4 loop, the x(4) is 14 and the best result is 19


19

x(0) is 14
After 1 loop, the x(1) is 19 and the best result is 19
After 2 loop, the x(2) is 20 and the best result is 20
After 3 loop, the x(3) is 14 and the best result is 20
After 4 loop, the x(4) is 19 and the best result is 20


20

* As we can see, the results are all right.

---